In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import numpy as np

# class FullSizeTestDataset(Dataset):
#     def __init__(self, directory):
#         self.directory = directory
#         self.image_files = [os.path.join(directory, f) for f in os.listdir(directory)]
#         self.transform = transforms.Compose([
#             transforms.ToTensor(),
#             # Add any other transformations here
#         ])

#     def __len__(self):
#         return len(self.image_files)

#     def __getitem__(self, idx):
#         img_path = self.image_files[idx]
#         img = Image.open(img_path)
#         img = img.resize((2048, 1536))  # Resize if necessary to handle edge cases
#         patches = self.image_to_patches(img)
#         return patches

#     def image_to_patches(self, img):
#         # Assuming img is a PIL image
#         patches = []
#         for i in range(0, img.width, 100):
#             for j in range(0, img.height, 100):
#                 # Handle edge cases for the last row and column
#                 width, height = min(100, img.width - i), min(100, img.height - j)
#                 patch = img.crop((i, j, i+width, j+height))
#                 patch = self.transform(patch)
#                 patches.append(patch)
#         return torch.stack(patches)  # Return a tensor of all patches


In [2]:
# Load your model
import torch
import numpy as np
from model import ResNeXt
from dataloader import get_dataloaders
from torchvision.utils import save_image
model = ResNeXt().to('cuda')
epoch = 10000

checkpoint = torch.load(f'checkpoints/model_epoch_{epoch}.pth')
model.load_state_dict(checkpoint['generator_state_dict'])

# Prepare DataLoader
batch_size = 50
sample_size = 100
train_loader, test_loader = get_dataloaders(batch_size=batch_size, sample_size=sample_size)
# test_dataset = FullSizeTestDataset('data/iphone/test_data/full_size_test_images')
# test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Evaluate
model.eval()
with torch.no_grad():
    for i, data in enumerate(test_loader, 0):
        real, target = data
        real = real.to('cuda')
        target = target.to('cuda')
        output = model(real)
        comparison = torch.cat((real[0], output[0]), dim=2)
        save_image(comparison, f'images/output_{i}.jpg')
        if i == 4:
            break


In [1]:
# Given probabilities
P_A = 0.5
P_B = 0.5
P_R_given_A = 0.1 * 0.4  # P(R|A) = P(R|open and A) * P(open|A)
P_R_given_B = 0.2 * 0.2  # P(R|B) = P(R|open and B) * P(open|B)

# Calculate P(A|R) using Bayes' theorem
P_A_given_R = (P_R_given_A * P_A) / (P_R_given_A * P_A + P_R_given_B * P_B)

print(f"The probability that a person who redeemed the coupon received email A is: {P_A_given_R:.2f}")

The probability that a person who redeemed the coupon received email A is: 0.50


In [1]:
from collections import deque

def processExecution(power, minPower, maxPower):
    n = len(power)
    ranges = list(zip(minPower, maxPower))
    num_ranges = len(ranges)
    result = [[0, 0] for _ in range(num_ranges)]
    
    queue = deque()
    curr_range = 0
    
    for val in power:
        # Add the current value to the queue
        queue.append(val)
        
        # Update the result for the current range
        while curr_range < num_ranges and val >= ranges[curr_range][0]:
            if val <= ranges[curr_range][1]:
                result[curr_range][0] += 1
                result[curr_range][1] += val
            curr_range += 1
        
        # Remove values from the queue that are outside the current range
        while queue and queue[0] < ranges[curr_range][0]:
            queue.popleft()
    
    return result

In [2]:
processExecution([11, 11, 11], [8,13], [11, 100])

[[1, 11], [0, 0]]